<h1 style="color:blue;text-align:center;">Lab4 - Part 2</h1>

<hr style="border:2px solid blue;">

<h3 style="color:blue;">Realised by: <strong style="color:black;">Ouahid Mariyam</strong></h3>


<h3 style="color:blue;">Guided by: <strong style="color:black;">Pr . ELAACHAk LOTFI</strong></h3>
   

<p><strong>Objective :</strong> The main purpose behind this lab is to get familiar with NLP language models
using Pytorch library.</p>


# Part 2: Transformer (Text generation) <a id='part-2'></a>

In [3]:
# Install the Hugging Face Transformers library from the GitHub repository
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-jq3ltjtm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jq3ltjtm
  Resolved https://github.com/huggingface/transformers.git to commit bdb9106f247fca48a71eb384be25dbbd29b065a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9128517 sha256=3cb41dcfc282f97ae41a65afd398693c0402eda955d6fa7f2ba206f7bbe7da4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-f37os1e4/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [1]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)  # Set logging level to CRITICAL to suppress warnings and info messages

import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Check if CUDA (GPU) is available and set the device accordingly
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
# Load the pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)  # Move the model to the selected device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# Function to select a random token ID from the top N tokens based on their probabilities
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) 
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [4]:
# Function to generate text of a specified length starting from an input string
def generate_some_text(input_str, text_len = 50):

    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)

    model.eval()
    with torch.no_grad():

        for i in range(text_len):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=10) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(output_text)

In [5]:
# Generate text starting from given input strings
generate_some_text('''i am just paranoid''')

i am just paranoid."<|endoftext|>By Michael Krieger

NEW YORK, May 17 (Reuters) - The United States has asked Russia to provide details about the number of U.S. personnel it has deployed to Syria, the State Department said on Friday, a


In [6]:
generate_some_text(" Artificial intelligence is ")

 Artificial intelligence is  being developed with great speed, and I would like to see this progress in a more realistic way.  This means, that AI systems are being designed to be as realistic as possible, while not breaking any laws of physics or mathematics. 


In [8]:
# Mount Google Drive to access data stored there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from torch.utils.data import Dataset, DataLoader
import os
import csv

# Define a custom dataset class for jokes
class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path='/content/drive/My Drive/Colab Notebooks/data/'):
        super().__init__()

        short_jokes_path = os.path.join(jokes_dataset_path, 'shortjokes.csv')

        self.joke_list = []
        self.end_of_text_token = ""

        if not os.path.exists(short_jokes_path):
            raise FileNotFoundError(f"The file {short_jokes_path} does not exist. Please check the path.")

        with open(short_jokes_path, newline='', encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            for row in csv_reader:
                joke_str = f"JOKE:{row[1]}{self.end_of_text_token}"
                self.joke_list.append(joke_str)

    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]

# Verify the dataset path and existence of the jokes file
dataset_path = '/content/drive/My Drive/Colab Notebooks/data/'
short_jokes_file = os.path.join(dataset_path, 'shortjokes.csv')

if not os.path.exists(short_jokes_file):
    raise FileNotFoundError(f"The file {short_jokes_file} was not found. Please ensure the file exists at the specified location.")

# Initialize dataset and dataloader
dataset = JokesDataset(jokes_dataset_path=dataset_path)
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Model training setup
BATCH_SIZE = 16
EPOCHS = 2
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400

from transformers import AdamW, get_linear_schedule_with_warmup
import torch

# Check and set the device
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# Prepare the model, optimizer, and scheduler for training
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

# Training loop would go here


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
print(os.getcwd())  # Print the current working directory
print(os.listdir('/content/drive/My Drive/Colab Notebooks/data/'))  # List the contents of the data directory

/content
['shortjokes.csv']


In [ ]:
# Training loop
for epoch in range(EPOCHS):
    print(f"EPOCH {epoch} started" + '=' * 30)
    for idx,joke in enumerate(joke_loader):
        # Logic to fit as many joke sequences into MAX_SEQ_LEN sequence as possible
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue

        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
            continue
        else:
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
            else:
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
                continue
        
        # Process the sequence through the model
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_joker_{epoch}.pt"))


models_folder = "trained_models"

EPOCH 0 started==============================
sum loss 6182.45166015625
sum loss 5950.6708984375
sum loss 5777.68310546875
sum loss 5648.5341796875
sum loss 5575.982421875
sum loss 5522.560546875
sum loss 5470.4365234375
sum loss 5434.81005859375
sum loss 5398.509765625
EPOCH 1 started==============================
sum loss 5358.80029296875
sum loss 5335.171875
sum loss 5312.67724609375
sum loss 5290.134765625
sum loss 5257.42529296875


In [1]:
# Load a specific model checkpoint
model_path = os.path.join(models_folder, f"gpt2_medium_joker_{1}.pt")
model.load_state_dict(torch.load(model_path))

# Path to save generated jokes
jokes_output_file_path = f'generated_{1}.jokes'

model.eval()
if os.path.exists(jokes_output_file_path):
    os.remove(jokes_output_file_path)

joke_num = 0
with torch.no_grad():
    for joke_idx in range(1000):
        joke_finished = False
        cur_ids = torch.tensor(tokenizer.encode("JOKE:")).unsqueeze(0).to(device)

        for i in range(100):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0, -1], dim=0)
            if i < 3:
                n = 20
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
            cur_ids = torch.cat([cur_ids, torch.ones((1, 1)).long().to(device) * next_token_id], dim=1)

            if next_token_id in tokenizer.encode(''):
                joke_finished = True
                break

        if joke_finished:
            joke_num += 1
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
            with open(jokes_output_file_path, 'a') as f:
                f.write(f"{output_text} \n\n")

NameError: name 'os' is not defined